In [1]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pytesseract
import itertools
import csv
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

""" image trasformation packages """
from PIL import Image
import skimage.io as skio
from skimage.util import dtype_limits
from skimage.morphology import label
from skimage.measure import regionprops
from skimage.filters import rank
from skimage.measure import compare_ssim, compare_mse

""" statistical data visualization packages"""
import seaborn as sns

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

In [2]:
from funcoes import checar_combinacoes, save_combinations, ler_letras, compare_images

# Gerar base de dados comparativa entre letras

Esta base deve ter os seguintes campos.

| resposta | char1           | char2              | MSE  | ISS | MSE_centro | ISS_centro |
|:------:|:-----------------:|:------------------:|:----:|:-----:|:---:|:---:|
|    0    | character234.Png  | character121.Png   | 1456 | 0.556 | bla | bla |
|	 1   | character411.Png  | character121.Png   | 1200 | 0.723 | bla | bla |
|	 0   | character234.Png  | character121.Png   | 1456 | 0.556 | bla | bla |
|	 0   | character234.Png  | character121.Png   | 1456 | 0.556 | bla | bla |

Nesta base, $p$ das linhas devem ser de mesmos caracteres e MESMA CAIXA (maiuscula ou minusc). Se a linha tem os mesmos caracteres ainda que em posições e formatos diferentes
a variável resposta vai receber o VALOR 1. Caso sejam caracteres diferentes vai receber 0.
Linhas com letra q minutos e Q maiusculo vão receber zero!

Fizemos um acréscimo onde teremos a adição das colunas abaixo:

| MSE_canny | ISS_canny| MSE_canny_centro| ISS_canny_centro| MSE_skeleton | ISS_skeleton | ISS_centro | MSE_skeleton_centro | ISS_skeleton_centro|
|:------:|:-----------------:|:------------------:|:----:|:-----:|:---:|:---:|:---:|:---:|
|    1456   | 1456  | 1456   | bla | bla | bla | bla | bla | bla |
|	 1200   |1200 | 0.723 | bla | bla | bla | bla | bla | bla |
|	 1456   | 1456 | 0.556 | bla | bla | bla | bla | bla | bla |
|	 1456   | 1456 | 0.556 | bla | bla | bla | bla | bla | bla |


In [3]:
letras, letters_dict = ler_letras("../letras.csv")
# retorna a permutação dois a dois da lista de arquivos de letras
letras_permutadas = list(itertools.combinations(letras, 2))

In [4]:
iguais, diferentes = checar_combinacoes(letras_permutadas, letters_dict)
print( "Temos {0} conjuntos iguais e {1} conjuntos com rótulos diferentes.".format(iguais,diferentes))
print( "A porcentagem atual é de: {0}".format( iguais/(iguais+diferentes) ))

Temos 5849.0 conjuntos iguais e 252991.0 conjuntos com rótulos diferentes.
A porcentagem atual é de: 0.022596971101838975


# Garatindo p% para cada classe

In [5]:
p = 30 # preencher com um valor inteiro entre 0 e 100
# 0 não irá realizar balanceamento

In [6]:
if p > 0:
    print( "Para garantir os " + str(p) + "% precisamos diminuir a quantidade de conjuntos, " + \
          "com rótulos diferentes, em {0}".format( round(diferentes - ((iguais / (p/100) )-iguais)) ))

Para garantir os 30% precisamos diminuir a quantidade de conjuntos, com rótulos diferentes, em 239343


In [7]:
if p > 0:
    k = round(diferentes - ((iguais / (p/100))-iguais))
    l = []
    for i, permuta in enumerate(letras_permutadas):
        if k <= 0:
            break
        if (letters_dict[permuta[0]]['rotulo'] != letters_dict[permuta[1]]['rotulo']):
            l.append(i)
            k = k - 1
    for index in sorted(l, reverse = True):
        del letras_permutadas[index]

In [8]:
if p > 0:
    iguais, diferentes = checar_combinacoes(letras_permutadas, letters_dict)
    print( "Temos {0} conjuntos iguais e {1} conjuntos com rótulos diferentes. Total = {2}".format(\
                                                            iguais,diferentes, iguais + diferentes))
    print( "A porcentagem atual é de: {0}%".format( round(iguais/(iguais+diferentes), 2)*100 ))

Temos 5849.0 conjuntos iguais e 13648.0 conjuntos com rótulos diferentes. Total = 19497.0
A porcentagem atual é de: 30.0%


## Gerando o arquivo de combinações

In [9]:
%%capture
if p > 0:
    arquivo = "../combinacoes" + str(p) + ".txt"
else:
    arquivo = "../combinacoes.txt"
save_combinations(letras_permutadas, letters_dict, arquivo = arquivo)

## Análises da base

In [10]:
#combinacoes = pd.read_csv("../combinacoes.txt")

In [11]:
#combinacoes[combinacoes.resposta == 1].mean()

In [12]:
#combinacoes[combinacoes.resposta == 0].mean()